In [684]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [685]:
df = pd.read_csv("Trial_dataset.csv",encoding='cp1252')
df

,Statement,Deceptive
0,No sir I did not. I absolutely did not. No sir...,1
1,"... and she approached me, and at that time th...",1
2,"No sir I was not, not at all.",1
3,"He had told me that he had had a dream that, a...",1
4,"And he told me that, ammm ï¿½ he was trying to...",1
...,...,...
115,"I was born in Durango, Colorado. My mother was...",0
116,"Um whenever I was little, um my oldest nephew ...",0
117,"We were married for two years, and um, we divo...",0
118,"Yes um, Reginald and I would have like long ta...",0


In [686]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [687]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [688]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df['clean_text'] = df['Statement'].apply(lambda x: finalpreprocess(x))
df.head()

,Statement,Deceptive,clean_text
0,No sir I did not. I absolutely did not. No sir...,1,sir absolutely sir sir
1,"... and she approached me, and at that time th...",1,approached time trip rug kind bump head nurser...
2,"No sir I was not, not at all.",1,sir
3,"He had told me that he had had a dream that, a...",1,told dream ammm forest kill laura help get rid...
4,"And he told me that, ammm ï¿½ he was trying to...",1,tell ammm ï½ try figure stuff ammm ï½ ask like...


In [689]:
# ASK IF  I SHOULD BRING BACK 'NO' AND 'NOT' FROM STOPWORDS????????????????????????????

In [690]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"],df["Deceptive"],test_size=0.2,shuffle=True)

In [691]:
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [692]:
#gives less weightage to less important words?
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
def fit(self, X, y):
        return self
def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [693]:
from sklearn import model_selection, naive_bayes, svm

In [694]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
#print(classification_report(y_test,y_predict))
#print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
print("NB Accuracy Score -> ",accuracy_score(y_predict,y_test )*100)

NB Accuracy Score ->  91.66666666666666


In [695]:
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       1.00      0.87      0.93        15

    accuracy                           0.92        24
   macro avg       0.91      0.93      0.91        24
weighted avg       0.93      0.92      0.92        24

Confusion Matrix: [[ 9  0]
 [ 2 13]]


In [696]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma=9)
SVM.fit(X_train_vectors_tfidf,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test_vectors_tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM,y_test )*100)

SVM Accuracy Score ->  91.66666666666666


In [697]:
print(classification_report(y_test,predictions_SVM))
print('Confusion Matrix:',confusion_matrix(y_test, predictions_SVM))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.93      0.93      0.93        15

    accuracy                           0.92        24
   macro avg       0.91      0.91      0.91        24
weighted avg       0.92      0.92      0.92        24

Confusion Matrix: [[ 8  1]
 [ 1 14]]


In [698]:
#import pickle


In [699]:
#Pkl_Filename = "Pickle_svm_Model.pkl"  

#with open(Pkl_Filename, 'wb') as file:  
 #   pickle.dump(SVM, file)

In [700]:
#with open(Pkl_Filename, 'rb') as file:  
 #   Pickled_svm_Model = pickle.load(file)

#Pickled_svm_Model

In [701]:
#score = Pickled_svm_Model.score(X_test_vectors_tfidf,y_test) 
# Print the Score

#print("Test score: {0:.2f} %".format(100 * score))

# Predict the Labels using the reloaded Model
#Ypredict = Pickled_svm_Model.predict(X_test_vectors_tfidf)  

#Ypredict

In [679]:
pip install joblib

In [680]:
import joblib

In [681]:
joblib_file = "joblib_svm_Model.pkl"  
joblib.dump(SVM, joblib_file)

['joblib_svm_Model.pkl']

In [682]:
joblib_svm_model = joblib.load(joblib_file)


joblib_svm_model

SVC(gamma=9, kernel='linear')

In [683]:
score = joblib_svm_model.score(X_test_vectors_tfidf,y_test)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = joblib_svm_model.predict(X_test_vectors_tfidf)  

Ypredict

Test score: 75.00 %


array([1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 1])